In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import heartpy as hp #이 모듈은 깔아야 import 할수 있음
import seaborn as sns
import scipy.stats as ss
import pywt

In [3]:
colname = ['ppg_grn', 'SKT', 'GSR', 'valence', 'arousal']
#data = pd.read_csv('WMinMaxs2e5.csv', header = None)
#data.columns = colname

MotherWavelet = pywt.Wavelet('db2')   # Mother wavelet (모함수) 지정
Level   = 4                    # Wavelet 분해 레벨 지정
select  = 4                    # 특징추출 영역 고주파 영역부터 개수 지정 (d1~)



# 함수 정의
def rms(x):
    return np.sqrt(np.mean(x**2))

def crestF(x):
    return np.max(x) / rms(x)

def shapeF(x):
    return rms(x) / np.mean(x)

def impulseF(x):
    return np.max(x)/ np.mean(x)



In [39]:
# 결과값 딕셔너리로 반환 함수
def NN_dictionary(working_data, time_interval, data1):
    RR_list = working_data['RR_list']
    RR_diff = working_data['RR_diff']
    cum = np.cumsum(RR_list)
    time = list(range(0, int(cum[-1]), time_interval))

    #PPG 시간영역(0~5)
    meanNN = []
    medianNN = []
    SDNN = []
    pNN50 = []
    NN50 = []
    RMSSD = []
    
    #PPG 주파수영역(6~16)
    maxFPPG = []
    minFPPG = []
    meanFPPG = []
    stdFPPG = []
    maxminFPPG = []
    kurFPPG = []
    skewFPPG = []
    rmsFPPG = []
    crestFPPG = []
    shapeFPPG = []
    impulseFPPG = []
    

    #GSR 시간영역(17~27)
    maxGSR = []
    minGSR = []
    meanGSR = []
    stdGSR = []
    maxminGSR =[]
    kurGSR = []
    skewGSR = []
    rmsGSR = []
    crestGSR = []
    shapeGSR = []
    impulseGSR = []

    #SKT 시간영역(28~38)
    maxSKT = []
    minSKT = []
    meanSKT = []
    stdSKT = []
    maxminSKT = []
    kurSKT = []
    skewSKT = []
    rmsSKT = []
    crestSKT = []
    shapeSKT = []
    impulseSKT = []

    #valence arousal labeling(39~42)
    label_valence = []
    label_arousal = []
    valence = []
    arousal = []

    for i in range(0, len(time) - 1):
        data1 = data.iloc[int(i*fs*time_interval/1000) : int((i+1)*fs*time_interval/1000), 0:5]
        idx = (cum >= time[i]) & (cum < time[i + 1])
        NN = RR_list[idx]
        diff = []
        for j in range(0, len(NN) - 1):
            diff.append(NN[j + 1] - NN[j])

        num50 = sum(np.array(diff) > 50)

        if len(diff) > 0:
            p50 = num50 / len(diff)
        else:
            p50 = 0
        rmssd = np.sqrt(np.mean(sum(NN ** 2)))

        meanNN.append(np.mean(NN))
        medianNN.append(np.median(NN))
        SDNN.append(np.std(NN))
        NN50.append(num50)
        pNN50.append(p50)
        RMSSD.append(rmssd)

        Coef1 = pywt.wavedec(data1['ppg_grn'], MotherWavelet, level=Level, axis=0)
        coef1 = Coef1[Level]
        
        maxFPPG.append(np.max(coef1))
        minFPPG.append(np.min(coef1))
        meanFPPG.append(np.mean(coef1))
        stdFPPG.append(np.std(coef1))
        maxminFPPG.append(np.ptp(coef1))
        kurFPPG.append(ss.kurtosis(coef1))
        skewFPPG.append(ss.skew(coef1))
        rmsFPPG.append(rms(coef1))
        crestFPPG.append(crestF(coef1))
        shapeFPPG.append(shapeF(coef1))
        impulseFPPG.append(impulseF(coef1))
        
        maxGSR.append(np.max(data1['GSR']))
        minGSR.append(np.min(data1['GSR']))
        meanGSR.append(np.mean(data1['GSR']))
        stdGSR.append(np.std(data1['GSR']))
        maxminGSR.append(np.ptp(data1['GSR']))
        kurGSR.append(ss.kurtosis(data1['GSR']))
        skewGSR.append(ss.skew(data1['GSR']))
        rmsGSR.append(rms(data1['GSR']))
        crestGSR.append(crestF(data1['GSR']))
        shapeGSR.append(shapeF(data1['GSR']))
        impulseGSR.append(impulseF(data1['GSR']))

        maxSKT.append(np.max(data1['SKT']))
        minSKT.append(np.min(data1['SKT']))
        meanSKT.append(np.mean(data1['SKT']))
        stdSKT.append(np.std(data1['SKT']))
        maxminSKT.append(np.ptp(data1['SKT']))
        kurSKT.append(ss.kurtosis(data1['SKT']))
        skewSKT.append(ss.skew(data1['SKT']))
        rmsSKT.append(rms(data1['SKT']))
        crestSKT.append(crestF(data1['SKT']))
        shapeSKT.append(shapeF(data1['SKT']))
        impulseSKT.append(impulseF(data1['SKT']))

        valence.append(np.mean(data1['valence']))
        arousal.append(np.mean(data1['arousal']))
        label_valence.append(int(np.mean(data1['valence'])*3/10.01)-1)
        label_arousal.append(int(np.mean(data1['arousal'])*3/10.01)-1)

    result = {'meanNN': meanNN,
              'medianNN': medianNN,
              'SDNN': SDNN,
              'pNN50': pNN50,
              'NN50': NN50,
              'RMSSD': RMSSD,  # ,'HR':HR

              'maxFPPG': maxFPPG,
              'minFPPG': minFPPG,
              'meanFPPG': meanFPPG,
              'stdFPPG': stdFPPG,
              'maxminFPPG': maxminFPPG,
              'kurFPPG': kurFPPG,
              'skewFPPG': skewFPPG,
              'rmsFPPG': rmsFPPG,
              'crestFPPG': crestFPPG,
              'shapeFPPG': shapeFPPG,
              'impulseFPPG': impulseFPPG,

              'maxGSR': maxGSR,
              'minGSR': minGSR,
              'meanGSR': meanGSR,
              'stdGSR': stdGSR,
              'maxminGSR': maxminGSR,
              'kurGSR': kurGSR,
              'skewGSR': skewGSR,
              'rmsGSR': rmsGSR,
              'crestGSR': crestGSR,
              'shapeGSR': shapeGSR,
              'impulseGSR': impulseGSR,

              'maxSKT': maxSKT,
              'minSKT': minSKT,
              'meanSKT': meanSKT,
              'stdSKT': stdSKT,
              'maxminSKT': maxminSKT,
              'kurSKT': kurSKT,
              'skewSKT': skewSKT,
              'rmsSKT': rmsSKT,
              'crestSKT': crestSKT,
              'shapeSKT': shapeSKT,
              'impulseSKT': impulseSKT,

              'valence': valence,
              'arousal': arousal,
              'label_valence': label_valence,
              'label_arousal': label_arousal

              }

    return result



In [41]:
"""
working_data, measures = hp.process(data['ppg_grn'].values, fs, calc_freq=True)
result = pd.DataFrame(NN_dictionary(working_data, time_interval, data))
print(result)
"""

#result.to_csv('timefeaturetest.csv', index = False)
#sns.pairplot(result)
#plt.show()

fs = 398
time_interval = 3000

Dir = 'C:/Users/ktk36/Desktop/Data_0813/W_IQR' #경로 설정
os.chdir(Dir)
filelist = os.listdir(Dir) #Dir 경로 폴더 안 모든 파일 이름 리스트

idx=0;
for file in filelist:
    data = pd.read_csv(file, header = None)
    data.columns = colname
    working_data, measures = hp.process(data['ppg_grn'].values, fs, calc_freq=False)
    result = pd.DataFrame(NN_dictionary(working_data, time_interval, data))
    result.to_csv('C:/Users/ktk36/Desktop/Data_0813/TotalWIQR/' +'total'+file, index = False)


BadSignalWarning: 
----------------
Could not determine best fit for given signal. Please check the source signal.
 Probable causes:
- detected heart rate falls outside of bpmmin<->bpmmax constraints
- no detectable heart rate present in signal
- very noisy signal (consider filtering and scaling)
If you're sure the signal contains heartrate data, consider filtering and/or scaling first.
----------------
